Juan Pablo Márquez 2022

In [1]:
# Instal·lem la llibreria whoosh que ens permetrà crear l'índex
!pip install whoosh

In [2]:
# per crear el nostre index importarem el paquet 'index'
from whoosh import index
# per poder emmagatzemar índex a un directori importarem la llibreria 'os'
import os
#per modelar l'esquema del nostre índex emprarem l'objecte 'Schema' 
from whoosh.fields import Schema
# a cada element del nostre esquema el definirem de tipus text.
from whoosh.fields import TEXT # També es poden desar tipus com NUMERIC, DATETIME, BOOLEAN

In [3]:
# Definim els 3 camps del nostre esquema: autor, títol i cos
# el paràmetre (stored=True) implica que s'emmagatzema el text original literalment
#sinó podria que es desés el text sense stopwords per exemple.
esquema = Schema(autor=TEXT(stored=True), 
                titol=TEXT(stored=True),
                cos=TEXT(stored=True),
                )

In [4]:
#crear un carpeta para desar índex
if not os.path.exists("directori_index"):
    os.mkdir("directori_index")
    
#crea índex (si existeix, el sobrescriu) amb l'esquema que hem definit abans
index.create_in("directori_index", esquema) 
#obre índex
index = index.open_dir("directori_index")
#crea un objete escritor para escriure a l'índex
writer = index.writer()

# Afegim a l'índex el document/s que tenguem (fer un bucle si en tenim més d'un!)
# vosaltres aquí haureu de recuperar les seccions de cada fitxer
writer.add_document(
                autor=u"Pepito Pérez",
                titol=u"El contingut de la matèria gris", #preprocesar la cadena si es considera necessari
                cos= u"Aquí posarem el contingut principal" )

# tancam l'escriptor i l'índex emmgatzemant tot el que hem escrit
writer.commit()

In [5]:
# 'qparser' ens permetra analitzar les querys (consultes) que ens introdueixin
from whoosh.qparser import *
#Definir un parser per un camp concret, és a dir, només cercaré en el camp 'cos'
parserCuerpo=QueryParser("cos", schema=esquema)
#Parsear la cadena que volem cercar per convertir-la a un objete consulta (query) format unicode
consulta=parserCuerpo.parse(u"contingut OR especialitzat OR hotels") #altres operadors són AND NOT
# el resultat és la consulta ja preparada per anar a l'índex
print (consulta)

(cos:contingut OR cos:especialitzat OR cos:hotels)


In [6]:
# si volem cercar en més d'un camp de l'esquema empram el constructor 'MultifieldParser'
parserMultiplesCamps = MultifieldParser(["titol", "cos"], schema=esquema)

#Parsear la cadena per convertir-la a un objete consulta (query)
consulta2=parserMultiplesCamps.parse(u"contingut OR especialitzat OR hotels")
print (consulta2)

(titol:contingut OR cos:contingut OR titol:especialitzat OR cos:especialitzat OR titol:hotels OR cos:hotels)


In [7]:
# importem 'scoring' per poder donar un pes d'importància als termes
from whoosh import scoring

# obrim índex i definim objecte 'searcher' per cercar
# definim el sistema de pesos amb TF_IDF
with index.searcher(weighting=scoring.TF_IDF()) as cercador:
    # Cerca dins índex els documents més semblants retornant un màxim de 20 documents (limit)
    # (terms = True) emmagatzema els termes que han fet match entre la consulta i el document
    # 'documents_recuperats' tendrà tots els documents que s'han considerat interessants
    # segons TD_IDF. A més s'ordenen els resultats per ordre d'importància, per defecte 
    # posa primer els que tenen la puntuació més alta.
    documents_recuperats = cercador.search(consulta2, limit=20, terms = True)
   
    #imprimir resultats
    for i in range(len(documents_recuperats)):
        print("El títol és: ",documents_recuperats[i]['titol'], 
              "\nEl cos és: ",documents_recuperats[i]['cos'],
             "\nLa puntuació és: ",str(documents_recuperats[i].score))

El títol és:  El contingut de la matèria gris 
El cos és:  Aquí posarem el contingut principal 
La puntuació és:  0.6137056388801094


In [8]:
# Per veure els termes de la consulta que han fet match amb els documents
if documents_recuperats.has_matched_terms():
    # Tots els termes que han fet match
    print('Termes en tota la colecció: ',documents_recuperats.matched_terms())

    # Si es vol veure per cada terme en cada document    
    for i, doc in enumerate(documents_recuperats, start = 1):
        print('Termes en doc', i, doc.matched_terms())

Termes en tota la colecció:  {('titol', b'contingut'), ('cos', b'contingut')}
Termes en doc 1 [('cos', b'contingut'), ('titol', b'contingut')]
